<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
hh_database = pd.read_csv('C:/Users/kkeri/Desktop/IT/Skillfactory/Projects/Анализ_резюме_из_HeadHunter/data/dst-3.0_16_1_hh_database.csv', sep=';') 

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
display(hh_database.head(3))
display(hh_database.tail(3))

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
print(hh_database.info()) 

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
display(hh_database.describe())

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
# Список возможных категорий образования, отсортированных по убыванию длины,
# чтобы избежать частичных совпадений (например, "среднее" не попало вместо "среднее специальное")
four_categories = [
    'неоконченное высшее',
    'среднее специальное',
    'высшее',
    'среднее'
]

# Извлекаем уровень образования из первых слов строки:
# проверяем, совпадает ли начало строки с какой-либо категорией
hh_database['Образование'] = hh_database['Образование и ВУЗ'].apply(
    lambda x: next(
        (
            category for category in four_categories
            if isinstance(x, str) and category == ' '.join(x.lower().split()[:len(category.split())])
        ),
        None  # Значение по умолчанию, если ни одна категория не найдена
    )
)

# Удаляем исходный столбец, так как уровень образования уже извлечён
hh_database = hh_database.drop(columns=['Образование и ВУЗ'])

print(hh_database['Образование'].unique()) 

display(hh_database.head(3)) 

print(f"Кол-во соискателей со средним образованием: {hh_database['Образование'].value_counts().get('среднее', 0)}")

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
# Извлекаем пол соискателя: 'М' для мужчин, 'Ж' для женщин
hh_database['Пол'] = hh_database['Пол, возраст'].apply(
    lambda x: 'М' if isinstance(x, str) and x.split()[0] == 'Мужчина'
    else 'Ж' if isinstance(x, str) and x.split()[0] == 'Женщина'
    else None
)

# Извлекаем возраст соискателя из строки, если это строка
hh_database['Возраст'] = hh_database['Пол, возраст'].apply(
    lambda x: int(x.split(',')[1].strip().split()[0]) if isinstance(x, str) else None
)

# Удаляем исходный столбец, так как данные уже извлечены
hh_database = hh_database.drop(columns=['Пол, возраст'])

display(hh_database.head(3)) 

print(f"Женские резюме: {round(hh_database['Пол'].value_counts(normalize=True).get('Ж', 0) * 100, 2)}%") 
print(f"Средний возраст соискателей: {round(hh_database['Возраст'].mean(), 1)}") 

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
# Возможные формы слов "год" и "месяц"
year_forms = ['год', 'года', 'лет']
month_forms = ['месяц', 'месяца', 'месяцев']

# Преобразуем опыт работы в месяцы
hh_database['Опыт работы (месяц)'] = hh_database['Опыт работы'].apply(
    lambda x: (
        sum(
            int(num) * 12 if unit in year_forms                                   # Умножаем количество лет на 12, если следующее слово — годовая форма
            else int(num) if unit in month_forms                                  # Иначе, если это месяцы — прибавляем просто int(num)
            else 0                                                                # Если ни год, ни месяц — возвращаем 0 (ничего не прибавляем)
            for num, unit in zip(x.split()[:7], x.split()[1:8])                   # Перебираем пары: число и следующее слово, ограничиваем длину, чтобы не захватывать лишние данные 
            if num.isdigit()                                                      # Обрабатываем только те слова, которые являются числами
        ) if isinstance(x, str) and x.strip().lower() != 'не указано' else np.nan # Обрабатываем только непустые строки, не равные "не указано" (без учёта регистра и лишних пробелов)
    )
)

hh_database = hh_database.drop(columns=['Опыт работы']) # Удаляем исходный столбец, так как данные уже извлечены

display(hh_database.head(3)) 

print(f"Медианный опыт работы (в месяцах): {hh_database['Опыт работы (месяц)'].median()}") 

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
# Список городов-миллионников, которые выделяются отдельно
million_cities = [
    'Новосибирск', 'Екатеринбург', 'Нижний Новгород',
    'Казань', 'Челябинск', 'Омск', 'Самара',
    'Ростов-на-Дону', 'Уфа', 'Красноярск',
    'Пермь', 'Воронеж', 'Волгоград'
]

# Создаем новый столбец 'Город' на основе столбца 'Город, переезд, командировки'
hh_database['Город'] = hh_database['Город, переезд, командировки'].apply(
    lambda x: 
        np.nan if pd.isna(x) # Если значение пропущено (NaN), оставляем NaN в новом столбце
        # Если значение строковое и первый элемент (до запятой) — один из ключевых городов,
        # то возвращаем его с удалением пробелов по краям
        else x.split(',')[0].strip()
        if isinstance(x, str)
        and x.split(',')[0].strip() in ['Москва', 'Санкт-Петербург'] + million_cities
        else 'другие' # Во всех остальных случаях присваиваем категорию 'другие'    
)

# Положительные ключевые фразы, указывающие на готовность к переезду
positive_relocation_phrases = [
    'готов к переезду', 'готова к переезду', 'хочу переехать', 'хочу переезжать',
    'могу переехать', 'готов переехать', 'готова переехать',
    'открыт к переезду', 'открыта к переезду',
    'рассматриваю переезд', 'не против переезда', 'не против переехать',
    'планирую переезд', 'ищу работу с переездом',
    'готов к релокации', 'готова к релокации', 'рассматриваю релокацию',
    'хочу релокацию', 'планирую релокацию',
    'готов переехать в другой город', 'готова переехать в другой город',
    'возможен переезд', 'возможна релокация',
    'готов к переезду в другие регионы', 'рассматриваю возможность переезда',
    'готов к работе в другом городе', 'релокация приветствуется',
    'не исключаю переезд', 'рассматриваю переезд при необходимости'
]

# Отрицательные ключевые фразы, указывающие на нежелание переезжать
negative_relocation_phrases = [
    'не готов к переезду', 'не готова к переезду', 'не хочу переехать',
    'не хочу переезжать', 'не рассматриваю переезд', 'не рассматриваю релокацию',
    'не готов к релокации', 'не готова к релокации',
    'не планирую переезд', 'не планирую релокацию',
    'против переезда', 'против релокации',
    'переезд не рассматриваю', 'релокация не интересует',
    'не рассматриваю смену города', 'не хочу менять город',
    'не хочу переезда', 'не готов переехать',
    'переезд невозможен', 'переезд не планирую',
    'не согласен на переезд', 'не согласна на переезд',
    'работа в другом городе не подходит', 'категорически против переезда'
]

# Создаем новый столбец 'Готовность к переезду' на основе текста из столбца
hh_database['Готовность к переезду'] = hh_database['Город, переезд, командировки'].apply(
    lambda x: np.nan 
    if pd.isna(x) # Если значение пропущено — оставляем NaN
    else (
        False if any(kw in x.lower() for kw in negative_relocation_phrases) # Есть отрицательные — False
        else True if any(kw in x.lower() for kw in positive_relocation_phrases) # Есть положительные фразы — True 
        else np.nan # Если ничего не найдено — NaN
    )
)

# Положительные ключевые фразы, указывающие на готовность к командировкам
positive_travel_phrases = [
    'готов к командировкам', 'готова к командировкам',
    'готов к редким командировкам', 'готова к редким командировкам',
    'готов к частым командировкам', 'готова к частым командировкам',
    'готов к длительным командировкам', 'готова к длительным командировкам',
    'готов к краткосрочным командировкам', 'готова к краткосрочным командировкам',
    'могу ездить в командировки', 'не против командировок',
    'рассматриваю командировки', 'планирую командировки',
    'не исключаю командировки', 'есть опыт командировок',
    'готов к разъездному характеру работы', 'готова к разъездному характеру работы',
    'разъездной характер работы подходит', 'разъездной график возможен',
    'разъезды не проблема', 'разъездной характер меня устраивает',
    'возможны командировки', 'могу ездить по стране', 'могу ездить по региону',
    'работа с выездами возможна', 'готов к выездам', 'готов к разъездам',
    'мобильный, готов к поездкам', 'мобильна, готова к поездкам',
    'рассматриваю работу с командировками', 'разъездная работа устраивает',
    'командировки приветствую', 'командировки возможны',
    'готов работать вне офиса'
]

# Отрицательные ключевые фразы, указывающие на нежелание ездить в командировки
negative_travel_phrases = [
    'не готов к командировкам', 'не готова к командировкам',
    'не хочу в командировки', 'не хочу ездить в командировки',
    'не рассматриваю командировки', 'против командировок',
    'командировки не интересуют', 'не планирую командировки',
    'командировки нежелательны', 'предпочитаю без командировок',
    'работа без командировок', 'без командировок',
    'разъезды не интересуют', 'не рассматриваю выезды',
    'не готов к разъездной работе', 'не готова к разъездной работе',
    'разъездной характер работы не подходит', 'разъездной характер нежелателен',
    'желательно без командировок', 'не хочу ездить',
    'не готов к частым поездкам', 'не планирую ездить',
    'не рассматриваю мобильную работу', 'не хочу выездную работу',
    'не готов к выездам', 'не готов к разъездам',
    'работа с выездами не подходит', 'поездки не устраивают',
    'непредпочтителен разъездной график', 'предпочитаю стационарную работу',
    'ищу работу без поездок', 'не согласен на командировки',
    'не согласна на командировки', 'категорически против командировок',
    'поездки невозможны', 'разъезды невозможны', 'откажусь от командировок'
]

# Создаем новый столбец 'Готовность к командировкам' на основе текста из столбца 
hh_database['Готовность к командировкам'] = hh_database['Город, переезд, командировки'].apply(
    lambda x: np.nan 
    if pd.isna(x) # Если значение пропущено — оставляем NaN
    else (
        False if any(kw in x.lower() for kw in negative_travel_phrases) # Есть отрицательные — False
        else True if any(kw in x.lower() for kw in positive_travel_phrases) # Есть положительные фразы — True 
        else False  # Если ничего не найдено — считаем False
    )
)

hh_database = hh_database.drop(columns=['Город, переезд, командировки']) # Удаляем исходный столбец, так как данные уже извлечены

display(hh_database.head(3))  

print(f"Соискатели в Санкт-Петербурге: {int(round(hh_database['Город'].value_counts(normalize=True).get('Санкт-Петербург', 0) * 100, 0))}%")
print(f"Соискатели готовые и к переездам, и к командировкам: {int(round((hh_database['Готовность к переезду'] & hh_database['Готовность к командировкам']).value_counts(normalize=True).get(True, 0) * 100, 0))}%") 

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
# Список вариантов желаемой занятости
employment_types = [
    'полная занятость', 'частичная занятость',
    'проектная работа', 'волонтерство', 'стажировка'
]

# Создаем по одному новому столбцу для каждой категории занятости
for emp_type in employment_types:
    hh_database[emp_type.capitalize()] = hh_database['Занятость'].apply(
        lambda x: np.nan if pd.isna(x)                                     # Если значение пропущено — оставляем NaN
        else emp_type in [s.strip().lower() for s in x.strip().split(',')] # Если категория присутствует в строке — значение True, иначе False
    )
    
hh_database = hh_database.drop(columns=['Занятость']) # Удаляем исходный столбец, так как данные уже извлечены

# Список вариантов желаемого графика работы 
schedule_types = [
    'полный день', 'сменный график', 'гибкий график', 
    'удалённая работа', 'вахтовый метод'
]

# Создаем по одному новому столбцу для каждой категории графика работы
for sch_type in schedule_types:
    hh_database[sch_type.capitalize()] = hh_database['График'].apply(
        lambda x: np.nan if pd.isna(x)                                     # Если значение пропущено — оставляем NaN
        else sch_type in [s.strip().lower() for s in x.strip().split(',')] # Если категория присутствует в строке — значение True, иначе False
    )
    
hh_database = hh_database.drop(columns=['График']) # Удаляем исходный столбец, так как данные уже извлечены

display(hh_database.head(3))  

print(f"Кол-во людей, ищущих проектную работу и волонтерство: {(hh_database['Проектная работа'] & hh_database['Волонтерство']).sum()}")
print(f"Кол-во людей, ищущих вахтовую работу и гибкий график: {(hh_database['Вахтовый метод'] & hh_database['Гибкий график']).sum()}") 

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# Читаем данные с курсами валют из CSV
exchange_rates = pd.read_csv(
    'C:/Users/kkeri/Desktop/IT/Skillfactory/Projects/Анализ_резюме_из_HeadHunter/data/ExchangeRates.csv',
    sep=','
)

# Преобразуем столбец с датой в формат datetime, оставляем только дату
exchange_rates['date'] = pd.to_datetime(
    exchange_rates['date'],
    format='%d/%m/%y',
    errors='coerce'
).dt.date

display(exchange_rates)

# Преобразуем столбец с датой обновления резюме, оставляем только дату
hh_database['Обновление резюме'] = pd.to_datetime(
    hh_database['Обновление резюме'],
    format='%d.%m.%Y %H:%M',
    errors='coerce'
).dt.date

# Словарь для сопоставления валют из резюме с ISO-кодами
currency_map = {
    'руб': 'RUB',
    'грн': 'UAH',
    'usd': 'USD',
    'eur': 'EUR',
    'белруб': 'BYN',
    'kgs': 'KGS',
    'сум': 'UZS',
    'azn': 'AZN',
    'kzt': 'KZT'
}

# Извлекаем из столбца "ЗП" числовую сумму и валюту (приводим к ISO)
hh_database[['Cумма ЗП', 'Валюта']] = hh_database['ЗП'].apply(
    lambda x: (
        int(''.join(c for c in x if c.isdigit())),                           # Извлекаем из строки только цифры
        currency_map.get(''.join(c for c in x if c.isalpha()).lower(), None) # Извлекаем из строки только буквы
    ) if isinstance(x, str) else (np.nan, np.nan)                                  
).apply(pd.Series) # Распаковка кортежа в отдельные столбцы DataFrame

# Объединяем таблицу с резюме и таблицу с курсами валют по дате и валюте
hh_database = hh_database.merge(
    exchange_rates[['currency', 'date', 'proportion', 'close']],
    how='left',
    left_on=['Обновление резюме', 'Валюта'],
    right_on=['date', 'currency']
)

# Для рублей заполняем курс (close) и пропорцию единицей
hh_database.loc[hh_database['Валюта'] == 'RUB', ['proportion', 'close']] = 1.0

# Вычисляем зарплату в рублях с учётом курса и пропорции
hh_database['ЗП (руб)'] = (hh_database['Cумма ЗП'] * hh_database['close'] / hh_database['proportion'])

# Удаляем исходный и промежуточные столбцы, так как данные уже извлечены
hh_database = hh_database.drop(columns=['ЗП', 'Cумма ЗП', 'Валюта', 'currency', 'date', 'proportion', 'close']) 
  
display(hh_database.head(3)) 

print(f"Желаемая медианная ЗП соискателей: {hh_database['ЗП (руб)'].median()} руб.") 

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# Графики
age_hist = px.histogram(
    hh_database,
    x='Возраст',
    labels={'Возраст': 'Возраст (лет)'},
    nbins=50,
    title='Распределение возраста соискателей',
    marginal='box' # добавим boxplot сверху 
)

age_hist.update_layout(bargap=0.1)
age_hist.show()

print(f"Мода: {hh_database['Возраст'].mode()[0]}")
print(f"Минимальный возраст: {hh_database['Возраст'].min()}")
print(f"Максимальный возраст: {hh_database['Возраст'].max()}")
print(f"Квантили:\n{hh_database['Возраст'].quantile([0.25, 0.5, 0.75])}")
print(f"Возможные аномалии: {hh_database[(hh_database['Возраст'] < 14) | (hh_database['Возраст'] > 80)].shape[0]}") 

# Сохранение графика в HTML
age_hist.write_html("visualization/age_hist.html") 

1. **Мода распределения** — 30 лет. Это наиболее часто встречающийся возраст среди соискателей, то есть у большинства кандидатов примерно такой возраст.

2. **Предельные значения:**

    * Минимальный возраст — 14 лет. Это минимальное значение, попавшее в данные, возможно, студенты или школьники с частичной занятостью.

    * Максимальный возраст — 100 лет, что можно считать выбросом или аномалией, т. к. очень редко кто в таком возрасте продолжает активный поиск работы.

3. **Интервал возраста большинства соискателей:**
По квартилям видно, что 50% соискателей имеют возраст от 27 до 36 лет (межквартильный размах, IQR). Медиана — 31 год, что соответствует средней возрастной группе специалистов.

4. **Аномалии:**
Обнаружена одна аномалия — возраст 100 лет. Однако, с учётом общего распределения, возраст 77 лет тоже достаточно высок и может считаться предельным значением, но не выбросом. Возраст 100 лет — явный выброс, который стоит проверить или исключить из анализа.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# Графики
exp_hist = px.histogram(
    hh_database,
    x='Опыт работы (месяц)',
    nbins=50,
    title='Распределение признака "Опыт работы"',
    marginal='box' # добавим boxplot сверху 
)

exp_hist.update_layout(bargap=0.1)
exp_hist.show()

print(f"Мода: {hh_database['Опыт работы (месяц)'].mode()[0]}")
print(f"Минимальный опыт: {hh_database['Опыт работы (месяц)'].min()}")
print(f"Максимальный опыт: {hh_database['Опыт работы (месяц)'].max()}")
print(f"Квантили:\n{hh_database['Опыт работы (месяц)'].quantile([0.25, 0.5, 0.75])}")
print(f"Возможные аномалии: {hh_database[(hh_database['Опыт работы (месяц)'] < 0) | (hh_database['Опыт работы (месяц)'] > 300)].shape[0]}") 

# Сохранение графика в HTML
exp_hist.write_html("visualization/experience_months_hist.html")

1. **Мода распределения** — 81 месяц. Это наиболее часто встречающееся значение опыта работы среди соискателей, то есть многие кандидаты имеют около 6.5 лет профессионального стажа.

2. **Предельные значения:**

    * Минимальный опыт — 1 месяц. Это минимальное значение в данных, что логично для начинающих специалистов или студентов.

    * Максимальный опыт — 1188 месяцев (~99 лет), что явно является выбросом и аномалией, скорее всего ошибкой или некорректной записью.

3. **Интервал опыта большинства соискателей:**
По квартилям видно, что 50% кандидатов имеют опыт работы от 57 до 154 месяцев (межквартильный размах, IQR). Медиана — 100 месяцев (~8.3 года), что соответствует зрелому профессиональному опыту.

4. **Аномалии:**
Обнаружено 1255 значений с аномально малым (меньше 0) или очень большим (больше 300 месяцев, то есть свыше 25 лет) опытом работы. Такие данные требуют дополнительной проверки или исключения из анализа, т. к. могут потенциально искажать статистику.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака заработной платы? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
# Графики без логарифмирования
sal_hist = px.histogram(
    hh_database,
    x='ЗП (руб)',
    nbins=50,
    title='Распределение заработной платы',
    marginal='box'
)

sal_hist.update_layout(bargap=0.1)
sal_hist.show()

hh_database_log = hh_database.copy()
hh_database_log['ЗП (руб) (логарифм)'] = np.log1p(hh_database_log['ЗП (руб)']) # Добавляем новый столбец с логарифмом зарплаты (используем log1p, чтобы избежать проблем с 0)

# Графики с логарифмированием
sal_hist_log = px.histogram(
    hh_database_log,
    x='ЗП (руб) (логарифм)',
    nbins=50,
    title='Распределение заработной платы (с применением логарифмирования)',
    marginal='box'
)

sal_hist_log.update_layout(bargap=0.1)
sal_hist_log.show()

print(f"Минимальная ЗП: {hh_database['ЗП (руб)'].min()}")
print(f"Максимальная ЗП: {hh_database['ЗП (руб)'].max()}")
print(f"Квантили:\n{hh_database['ЗП (руб)'].quantile([0.25, 0.5, 0.75])}")
print(f"Возможные аномалии: {hh_database[(hh_database['ЗП (руб)'] < 5000) | (hh_database['ЗП (руб)'] > 300_000)].shape[0]}") 

# Сохранение графиков в HTML
sal_hist.write_html("visualization/salary_raw_hist.html")     # Без логарифмирования
sal_hist_log.write_html("visualization/salary_log_hist.html") # С логарифмированием

**Примечание:** Использование логарифмирования позволило уменьшить влияние экстремально больших значений и получить более читаемую гистограмму. Boxplot сверху показывает медиану, межквартильный размах и выбросы, которые легко визуально определить.

1. **Предельные значения признака:**

    * Минимальная зарплата — 1 рубль. Вероятно, это ошибка или некорректные данные, поскольку зарплата в 1 рубль практически невозможна.

    * Максимальная зарплата — 24 304 876 рублей. Это явно выброс, скорее всего опечатка или экстремальный случай.

2. **Интервал заработной платы большинства соискателей:** Квантили показывают, что 25% соискателей хотят зарплату до 37 082 руб, 50% — до 59 019 руб (медиана), 75% — до 95 000 руб. Это означает, что основная масса желаемой зарплаты находится примерно в диапазоне 37 000 — 95 000 рублей.

4. **Аномалии:** По заданным границам аномалиями считаются зарплаты ниже 5000 рублей и выше 300 000 рублей — таких записей 470. Среди них скорее всего есть как реальные ошибки и опечатки (например, зарплата 1 рубль), так и высокие значения зарплат, которые могут быть реальными, но редкими.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
filtered_data = hh_database[hh_database['ЗП (руб)'] < 1_000_000] # Отбираем только ЗП < 1 млн

median_salary = filtered_data.groupby('Образование', as_index=False)['ЗП (руб)'].median() # Группируем по образованию и считаем медиану
median_salary.rename(columns={'ЗП (руб)': 'Медианная ЗП'}, inplace=True) # Переименование столбца

# График
fig = px.bar(
    median_salary,
    x='Образование',
    y='Медианная ЗП',
    text='Медианная ЗП', # Цифры ЗП над столбцами 
    title='Медианная ЗП по уровням образования',
    labels={'Образование': 'Уровень образования', 'Медианная ЗП': 'Медианная ЗП (руб)'}
)

fig.update_traces(
    textposition='outside', # Размещение текста над столбцом
    textfont_size=14
).update_layout(
    height=600,
    width=800
)

fig.show() 

# Сохранение графика в HTML
fig.write_html("visualization/median_salary_by_education.html")

1. **Наибольшая медианная зарплата** наблюдается у соискателей с высшим образованием — медианная ЗП составляет 60 000 рублей. Это логично, так как кандидаты с высшим образованием чаще претендуют на более квалифицированные и оплачиваемые должности.

2. **Средняя медианная зарплата** у соискателей с неоконченным высшим образованием — 50 000 рублей. Это чуть ниже, чем у тех, кто завершил обучение, но выше, чем у соискателей без высшего образования.

3. **Наименьшая медианная зарплата** зафиксирована у соискателей со средним и средне-специальным образованием — по 40 000 рублей. Это самые низкие значения среди всех групп.

**Вывод:** мы чётко видим положительную корреляцию между уровнем образования и медианной желаемой зарплатой:
чем выше образование — тем в среднем выше зарплатные ожидания.



5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
filtered_data = hh_database[hh_database['ЗП (руб)'] < 1_000_000] # Отбираем только ЗП < 1 млн

# График
fig = px.box(
    filtered_data,
    x='Город',
    y='ЗП (руб)',
    title='Распределение ЗП по городам',
    labels={'ЗП (руб)': 'Желаемая зарплата (руб)'}
)

fig.update_layout(height=1200, width=1600)
fig.show() 

# Сохранение графика в HTML
fig.write_html("visualization/salary_distribution_by_city.html")

1. Медианные уровни заработной платы не сильно варьируются в зависимости от города, но медиана в Москве заметно выше, чем в остальных городах (немного выше также Санкт-Петербург). Это отражает реальную рыночную ситуацию: работодатели в столице предлагают более высокие оклады.

2. Размах зарплат в Москве также сильно различается.
В Москве разброс значений широкий: есть как высокие, так и низкие ожидания. В других городах зарплатные ожидания более сгруппированы (коробка уже). 

3. Признак города — важный фактор для прогнозирования зарплаты.
Различия в медиане и диапазоне Москвы от других городов говорит о том, что столица существенно влияет на желаемый уровень ЗП.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
median_salary = hh_database.groupby(['Готовность к переезду', 'Готовность к командировкам'], as_index=False)['ЗП (руб)'].median() # Группируем и считаем медиану 
median_salary.rename(columns={'ЗП (руб)': 'Медианная ЗП'}, inplace=True) # Переименование столбца 

# Создаем новые столбцы на основе других столбцов
# В новых столбцах значения True и False заменяются на понятные строки
median_salary['Переезд'] = median_salary['Готовность к переезду'].replace({True: 'Готов к переезду', False: 'Не готов к переезду'})
median_salary['Командировки'] = median_salary['Готовность к командировкам'].replace({True: 'Готов к командировкам', False: 'Не готов к командировкам'})

# График
fig = px.bar(
    median_salary,
    x='Переезд',
    y='Медианная ЗП',
    color='Командировки',
    barmode='group',
    text='Медианная ЗП', # Цифры ЗП над столбцами 
    title='Медианная ЗП по готовности к переезду и командировкам',
    labels={'Медианная ЗП': 'Заработная плата (руб)'}
)

fig.update_traces(textposition='outside')
fig.update_layout(height=600, width=1000)
fig.show() 

# Сохранение графика в HTML
fig.write_html("visualization/median_salary_by_relocation_and_travel.html")

1. **Наименьшая медианная ЗП — 40 000 руб.** у соискателей, которые **не готовы ни к переезду, ни к командировкам**. Это может объясняться ограниченной мобильностью и, как следствие, меньшим выбором вакансий или низкой готовностью работать в условиях гибкости.

2. **Готовность к командировкам при отсутствии готовности к переезду** повышает медианную зарплату до 60 000 руб. Это говорит о том, что даже частичная мобильность уже ценится работодателями и отражается на уровне оплаты.

3. **Готовность к переезду, но нежелание ездить в командировки** даёт медианную зарплату 50 000 руб., что выше, чем у полностью «неготовых» кандидатов, но ниже, чем у тех, кто готов к командировкам, следовательно, переезд не всегда напрямую влияет на зарплату.

4. **Максимальная медианная ЗП — около 65 849 руб. — у тех, кто готов и к переезду, и к командировкам.** Это отражает высокую мобильность и готовность к разным условиям работы, что обычно востребовано и оплачивается выше.

**Вывод:** признак готовности к переезду и командировкам стоит учитывать при прогнозировании заработной платы, поскольку он отражает важный аспект гибкости кандидата и потенциально влияет на возможности трудоустройства и уровень оплаты. Разница в медианных значениях между категориями показывает, что работодатели ценят мобильность и компенсируют её более высокой оплатой.



7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
# Сводная таблица
pivot = hh_database.pivot_table(
    values='ЗП (руб)',
    index='Образование',
    columns='Возраст',
    aggfunc='median',
    fill_value=0
)

# Тепловая карта
fig = px.imshow(
    pivot,
    color_continuous_scale='YlGnBu',
    aspect='auto',
    labels=dict(x='Возраст (лет)', y='Образование', color='Медианная ЗП'),
    title='Медианная ЗП по возрасту и уровню образования'
)

fig.update_layout(width=1750, height=400)
fig.show() 

# Сохранение графика в HTML
fig.write_html("visualization/median_salary_by_age_and_education.html")

**Зависимость зарплаты от образования:**

* Разрыв между высшим / неоконченным высшим и средним / средним специальным уровнями образования особенно заметен с 40 лет, когда достигается пик карьеры.

* Среднее образование демонстрирует наиболее быстрый рост зарплаты с возрастом по сравнению с другими уровнями образования, однако итоговые значения остаются ниже.

**Зависимость от возраста:**

* У соискателей с высшим образованием заработная плата растёт примерно до 45 лет, после чего чаще всего начинает постепенно снижаться.

* У молодых специалистов (18–25 лет) медианные ЗП низкие, за исключением группы с средним образованием, где нет резко выраженного роста с возрастом.

**Вывод:**

* Признаки "Возраст" и "Образование" действительно играют ключевую роль в формировании желаемого уровня заработной платы.

* Высшее и неоконченное высшее образование дают существенные преимущества в перспективе, особенно на зрелом этапе карьеры.

* Возрастная зависимость нелинейна: доходы сначала растут, затем достигают плато или начинают снижаться.


8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
scatter_data = hh_database.copy()
scatter_data['Опыт работы (год)'] = scatter_data['Опыт работы (месяц)'] / 12 # Переводим опыт работы из месяцев в годы

# График рассеяния
fig = px.scatter(
    data_frame=scatter_data,
    x='Возраст',
    y='Опыт работы (год)',
    title='Зависимость опыта работы от возраста',
    labels={
        'Возраст': 'Возраст (лет)',
        'Опыт работы (год)': 'Опыт работы (лет)'
    }
)

# Добавляем диагональную линию y = x (опыт = возраст)
fig.add_shape(
    type='line',
    x0=0, y0=0,
    x1=100, y1=100,
    line=dict(color='red', dash='dash'),
)

fig.update_layout(height=600, width=800)
fig.show() 

print(f"Количество точек строго выше прямой y = x: {scatter_data[scatter_data['Опыт работы (год)'] > scatter_data['Возраст']].shape[0]}") 

# Сохранение графика в HTML
fig.write_html("visualization/experience_vs_age_scatter.html")

**Вывод:** почти все точки лежат ниже линии y = x — значит, возраст больше опыта, что логично. Только 7 точек выше линии — это аномалии (опыт > возраст) и требуют чистки. Возраст и опыт положительно коррелируют, но разброс заметен, особенно у взрослых.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# Сводная таблица
pivot = hh_database.pivot_table(
    values='ЗП (руб)',
    index='Пол',
    columns='Возраст',
    aggfunc='median',
    fill_value=0
)

# Тепловая карта
fig = px.imshow(
    pivot,
    color_continuous_scale='YlGnBu',
    aspect='auto',
    labels=dict(x='Возраст (лет)', y='Пол', color='Медианная ЗП'),
    title='Медианная ЗП по возрасту и половой принадлежности'
)

fig.update_layout(width=1750, height=400)
fig.show() 

# Сводная таблица
pivot = hh_database.pivot_table(
    values='ЗП (руб)',
    index='Пол',
    columns='Образование',
    aggfunc='median',
    fill_value=0
)

# Тепловая карта
fig = px.imshow(
    pivot,
    color_continuous_scale='RdPu',  
    aspect='auto',
    labels=dict(x='Образование', y='Пол', color='Медианная ЗП'),
    title='Медианная ЗП по уровню образования и половой принадлежности'
)

fig.update_layout(width=1000, height=400)
fig.show() 

# Сохранение графиков в HTML
fig.write_html("visualization/median_salary_by_age_and_gender.html")       # ЗП по возрасту и полу
fig.write_html("visualization/median_salary_by_education_and_gender.html") # ЗП по образованию и полу 

**Вывод:**

1. Мужчины получают выше ЗП в среднем на всех уровнях образования и возрастов.

2. Зарплата женщин достигает потолка раньше и остаётся стабильной, но низкой.

3. Образование влияет на ЗП, но не нивелирует гендерный разрыв.

4. Женщины реже представлены в выборке пожилого возраста, мужчины — активны дольше на рынке труда.

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
duplicates = hh_database[hh_database.duplicated(
    subset=hh_database.columns, # Проверка дубликатов по всем колонкам
    keep='first'                # Первое вхождение строки не считается дубликатом
)]
print(f"Кол-во полных дубликатов: {duplicates.shape[0]}")
hh_database = hh_database.drop_duplicates() 

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
null_data = hh_database.isnull().sum()
print(null_data[null_data > 0])

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# Удаляем строки с пропусками в столбцах "Последнее/нынешнее место работы" и "Последняя/нынешняя должность"
hh_database = hh_database.dropna(subset=[
    'Последнее/нынешнее место работы', 
    'Последняя/нынешняя должность'
]) 

# Заполняем пропуски в столбце "Опыт работы (месяц)" медианным значением этого столбца
hh_database['Опыт работы (месяц)'] = hh_database['Опыт работы (месяц)'].fillna(hh_database['Опыт работы (месяц)'].median()) 

print(f"Результирующее среднее значение в столбце «Опыт работы (месяц)»: {round(hh_database['Опыт работы (месяц)'].mean())}") 

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
before = hh_database.shape[0]
hh_database = hh_database.drop(hh_database[(hh_database['ЗП (руб)'] > 1_000_000) | (hh_database['ЗП (руб)'] < 1_000)].index)
after = hh_database.shape[0]

print(f"Удалено выбросов: {before - after}") 

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
before = hh_database.shape[0]
hh_database = hh_database.drop(hh_database[(hh_database['Опыт работы (месяц)'] / 12) > hh_database['Возраст']].index)
after = hh_database.shape[0]

print(f"Удалено выбросов: {before - after}") 

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
def detect_outliers_zscore(
    data, 
    column, 
    left_sigma=3, 
    right_sigma=3, 
    log_transform=True, 
    plot=True
):
    """
    Выявление выбросов с помощью Z-отклонения (с асимметричными границами).

    Parameters:
    - data: pd.DataFrame — исходный DataFrame
    - column: str — имя признака
    - left_sigma: int/float — левая граница (обычно 3)
    - right_sigma: int/float — правая граница (обычно 3)
    - log_transform: bool — использовать логарифмирование или нет
    - plot: bool — строить график или нет

    Returns:
    - outliers: pd.DataFrame — выбросы
    """
        
    # Создаём новый объект Series с логарифмированными значениями (используем log1p, чтобы избежать проблем с 0)
    x = np.log1p(data[column]) if log_transform else data[column]

    # Параметры распределения
    mean = x.mean()
    std = x.std()

    # Границы 
    left = mean - left_sigma * std
    right = mean + right_sigma * std 
    
    # Выбросы 
    outliers = data[(x < left) | (x > right)]

    if plot:
        # Построение графика
        plt.figure(figsize=(12, 6))
        histplot = sns.histplot(x, bins=50, kde=True)

        histplot.axvline(mean, color='k', lw=2, label='Среднее')                         # Среднее значение
        if left_sigma != 3:
            histplot.axvline(mean - 3 * std, color='red', linestyle='--', label='-3σ')   # Левая граница -3σ
        if right_sigma != 3:
            histplot.axvline(mean + 3 * std, color='blue', linestyle='--', label='+3σ')  # Правая граница +3σ
        histplot.axvline(left, color='purple', linestyle='-.', label=f'-{left_sigma}σ')  # Левая граница произвольная
        histplot.axvline(right, color='green', linestyle='-.', label=f'+{right_sigma}σ') # Правая граница произвольная

        plt.title(f'Распределение признака "{column}"{" (с применением логарифмирования)" if log_transform else ""}') 
        plt.xlabel(f'{column} (логарифм)' if log_transform else column)
        plt.ylabel('Количество')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    return outliers    


outliers = detect_outliers_zscore(
    hh_database,
    column='Возраст',
    left_sigma=3,
    right_sigma=4,
    log_transform=True,
    plot=True
)

print(f"Кол-во выбросов: {outliers.shape[0]}")
print(f"Минимальный возраст среди выбросов: {outliers['Возраст'].min()} лет")
display(outliers[['Возраст']])

hh_database = hh_database.drop(outliers.index) # Удаляем выбросы 

1. **Асимметрия вправо (положительная скошенность):**

    * Распределение **не симметрично**.

    * Левая часть более резкая и крутая, правая часть — длинный пологий хвост.

    * Это говорит о том, что **в выборке много молодых соискателей**, а **пожилые встречаются реже, но могут быть с сильно выбивающимися значениями**.

2. **Среднее значение смещено в сторону правого хвоста:**

    * Чёрная линия (среднее) ближе к «длинному» правому краю, а не посередине.

    * Это характерно для скошенных распределений.

3. **Интервал ±3σ охватывает почти все данные:**

    * Красная линия (−3σ) почти не затрагивает данные — слева почти нет значений.

    * Синяя линия (+3σ) отсекает правый край — после неё уже совсем редкие значения.

    * Это позволяет использовать метод трёх сигм для поиска выбросов (например, людей с экстремально большим возрастом).

**Вывод:** после логарифмирования возраст по-прежнему распределён не нормально — наблюдается положительная асимметрия.
Это означает, что данные всё ещё содержат хвост соискателей старшего возраста, хоть логарифмирование и уменьшило его влияние.
Метод трёх сигм помогает выделить этих крайних пользователей — потенциальные выбросы по возрасту.